# Transformer

Вспомним про encoder-часть трансформера


<img src="images/transformer.png"/>


1. Кодируем все слова эмбедингами


<img src="images/transec1.png"/>


2. Добавляем позиции слов
3. Энкодер

<img src="images/transec2.png"/>


3.1 Первая часть - MultiHead attention. Нужен, чтобы искать взаимосвязь между входными словами

<img src="images/transec3.png"/>

Например, query - ваш запрос в поисковой строке, key - какое-то описание объекта,с которым взаимодействовали (title страницы), values - драфт, из которого вы выбираете.

3.2 Результат self-attention шкалирует и преобразуем софтмаксом. В случае с простым текстом получим матрицу распределения "вероятностей взаимосвязи" слов

Дальше просто много слоев

# BERT

BERT (Bidirectional Encoder Representations from Transformers) 


Идея в основе BERT лежит очень простая: давайте на вход нейросети будем подавать фразы, в которых 15% слов заменим на [MASK], и обучим нейронную сеть предсказывать эти закрытые маской слова.


А для того, чтобы нейросеть научилась понимать соотношения между разными предложениями, дополнительно обучим ее предсказывать, является ли вторая фраза логичным продолжением первой. Или это какая-то случайная фраза, не имеющая никакого отношения к первой.


Так, для двух предложений: "Я пошел в магазин." и "И купил там молоко.", нейросеть должна ответить, что это логично. А если вторая фраза будет "Карась небо Плутон", то должна ответить, что это предложение никак не связано с первым. Ниже мы поиграемся с обоими этими режимами работы BERT.


Обучив таким образом нейронную сеть на корпусе текстов из Wikipedia и сборнике книг BookCorpus в течении 4 дней на 16 TPU, получили BERT.


<img src="images/bert.png"/>


Как это делается на практике:


<img src="images/bertTrain.png"/>


Кодирует 15 процентов слов [MASK] и учим трансформеры + fully connected слои для классификации. Далее, формируем сет, где 50% предложений идут последовательно и 50% рандомно. Учим классификатор (заменяем первый и последний токены предложения, чтобы модель могла их отличить). Учится софтмакс


<img src="images/bertTrain2.png"/>


In [1]:
config_file = 'bert_data/bert_config.json'
bert_model = 'bert_data/bert_model.ckpt'
vocab_file = 'bert_data/vocab.txt'

In [2]:
from bert_token import FullTokenizer, printable_text
from keras_bert import load_trained_model_from_checkpoint

tokenizer = FullTokenizer(vocab_file=vocab_file, do_lower_case=False)
model = load_trained_model_from_checkpoint(config_file, bert_model, training=True)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 512)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
______________________________________________________________________________________________

# Предобученный берт для предсказания пропущенных слов

In [3]:
import re

phrase = 'Сегодня хорошая [MASK] и поэтому собака [MASK].'

phrase = re.sub("\s*\[MASK\]\s*", "[MASK]", phrase)
print(phrase)
phrase = phrase.split('[MASK]')
tokens = ['[CLS]']
for i in range(len(phrase)):
    if i == 0:
        tokens = tokens + tokenizer.tokenize(phrase[i]) 
    else:
        tokens = tokens + ['[MASK]'] + tokenizer.tokenize(phrase[i]) 
tokens = tokens + ['[SEP]'] 

Сегодня хорошая[MASK]и поэтому собака[MASK].


In [4]:
tokens

['[CLS]',
 'Се',
 '##го',
 '##дня',
 'хор',
 '##ош',
 '##ая',
 '[MASK]',
 'и',
 'поэтому',
 'со',
 '##бак',
 '##а',
 '[MASK]',
 '.',
 '[SEP]']

In [5]:
import numpy as np

def token_converter(tokens):
    token_input = tokenizer.convert_tokens_to_ids(tokens) 
    token_input = token_input + [0] * (512 - len(token_input))

    mask_input = [0]*512
    for i in range(len(mask_input)):
        if token_input[i] == 103:
            mask_input[i] = 1
    seg_input = [0]*512

    token_input = np.asarray([token_input])
    mask_input = np.asarray([mask_input])
    seg_input = np.asarray([seg_input])
    
    return token_input, mask_input, seg_input

token_input, mask_input, seg_input = token_converter(tokens)

predicts = model.predict([token_input, seg_input, mask_input])[0] 
predicts = np.argmax(predicts, axis=-1)
predicts = predicts[0][:len(tokens)]

In [6]:
out = []
for i in range(len(mask_input[0])):
    if mask_input[0][i] == 1:                       
        out.append(predicts[i]) 

out = tokenizer.convert_ids_to_tokens(out) 
print(out)
out = ' '.join(out)                               
out = printable_text(out)            
out = out.replace(' ##','') 
out

[',', 'А']


', А'

# ПРедобученный берт для предсказания последовательности 2 фраз

In [7]:
tokens_sen_1 = tokenizer.tokenize("светит яркое солнце")
tokens_sen_2 = tokenizer.tokenize("пиво закончилось")

tokens = ['[CLS]'] + tokens_sen_1 + ['[SEP]'] + tokens_sen_2 + ['[SEP]']

In [8]:
tokens

['[CLS]',
 'свет',
 '##ит',
 'я',
 '##рко',
 '##е',
 'сол',
 '##нце',
 '[SEP]',
 'п',
 '##иво',
 'закончил',
 '##ось',
 '[SEP]']

In [9]:
token_input, mask_input, seg_input = token_converter(tokens)

In [10]:
predicts = model.predict([token_input, seg_input, mask_input])[1] 
predicts[0][0]

0.98013216

# Fine-tuning bert

In [11]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("data/train.csv")
df_val = pd.read_csv("data/val.csv")

In [12]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [13]:
train_tokens = map(tokenizer.tokenize, df_train['text'])
train_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], train_tokens)
train_token_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
train_token_ids = map(lambda tids: tids + [0] * (512 - len(tids)), train_token_ids)
train_token_ids = np.array(list(train_token_ids))

test_tokens = map(tokenizer.tokenize, df_val['text'])
test_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], test_tokens)
test_token_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

test_token_ids = map(lambda tids: tids + [0] * (512 - len(tids)), test_token_ids)
test_token_ids = np.array(list(test_token_ids))

train_labels_final = df_train['class'].values
test_labels_final = df_val['class'].values

In [14]:
#!pip install bert-for-tf2

In [15]:
import tensorflow as tf
import bert

tf.config.set_visible_devices([], 'СPU')

bert_params = bert.params_from_pretrained_ckpt("bert_data")
bert_layer = bert.BertModelLayer.from_params(bert_params)
bert_layer.apply_adapter_freeze()
    
model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(512,), dtype='int32'),
        bert_layer,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])

model.build(input_shape=(None, 512))
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.00001), metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert_model_layer (BertModelL (None, 512, 768)          177261312 
_________________________________________________________________
flatten (Flatten)            (None, 393216)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               100663552 
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 277,925,121
Trainable params: 277,925,121
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
with tf.device('/cpu:0'):
    history = model.fit(
            train_token_ids,
            train_labels_final,
            epochs=5,
            validation_data=(test_token_ids, test_labels_final),
            verbose=1
        )

Epoch 1/5


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[32,512,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential/bert_model_layer/encoder/layer_11/output/dropout_36/dropout/Mul (defined at C:\ProgramData\python\lib\site-packages\bert\transformer.py:66) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[gradient_tape/sequential/bert_model_layer/encoder/layer_1/output/LayerNorm/mul/Reshape/_854]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[32,512,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential/bert_model_layer/encoder/layer_11/output/dropout_3

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

Error: Kernel is dead